<a href="https://colab.research.google.com/github/DAVOR2003/ADA/blob/main/CPMPv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Layout functions

In [77]:
def compute_sorted_elements(stack):
    if len(stack)==0: return 0
    sorted_elements=1
    while(sorted_elements<len(stack) and stack[sorted_elements] <= stack[sorted_elements-1]):
        sorted_elements +=1
    
    return sorted_elements   
    
class Layout:
    def __init__(self, stacks, H):
        self.stacks = stacks
        self.sorted_elements = []
        self.total_elements = 0
        self.sorted_stack = []
        self.unsorted_stacks = 0
        self.steps = 0
        self.current_step = 0
        self.moves = []
        self.H = H
        self.full_stacks = 0
        self.last_sd = None
        self.bsg_moves=[]
        self.prev_move=None
        self.dismantling_stack=None
        self.max_item=max(set().union(*stacks))
        self.G=0
        
        j=0
        
        for stack in stacks:
            if len(stack)>0:
              g = max(stack)
              if self.G<g: self.G=g

            self.total_elements += len(stack)
            if len(stack) == self.H: self.full_stacks+=1
            self.sorted_elements.append(compute_sorted_elements(stack))
            if not self.is_sorted_stack(j):
                self.unsorted_stacks += 1
                self.sorted_stack.append(False)
            else: self.sorted_stack.append(True)
            j += 1
    
    
    
    def move(self,move):
        i = move[0]; j=move[1]
        
        if i==j: return None
        if len(self.stacks[i]) == 0: return None
        if len(self.stacks[j]) == self.H: return None
        
        if len(self.stacks[i]) == self.H: self.full_stacks -= 1
        if len(self.stacks[j]) == self.H-1: self.full_stacks += 1
        
        c = self.stacks[i][-1]

        if self.is_sorted_stack(i):
            self.sorted_elements[i] -= 1
            self.prev_move = "g"
        else:  self.prev_move ="b"
            
        if self.is_sorted_stack(j) and self.gvalue(j) >= c:
            self.sorted_elements[j] += 1
            self.prev_move += "g"
        else: self.prev_move += "b"
            
        self.stacks[i].pop(-1)
        self.stacks[j].append(c)
        
        self.is_sorted_stack(i)
        self.is_sorted_stack(j)
        self.steps += 1
        self.current_step += 1
        self.moves.append((i,j))
        
        return c
                       
    def is_sorted_stack(self, j):
        sorted = len(self.stacks[j]) == self.sorted_elements[j]

        if j<len(self.sorted_stack) and self.sorted_stack[j] != sorted: 
            self.sorted_stack[j] = sorted
            if sorted == True: self.unsorted_stacks -= 1
            else: self.unsorted_stacks += 1
        return sorted

    def gvalue(self, i):
        if len(self.stacks[i])==0: return self.G
        else: return self.stacks[i][-1]

def reachable_height(layout, i):
    if not layout.is_sorted_stack(i): return -1;
    
    top = layout.gvalue(i)
    h = len(layout.stacks[i])
    if h==layout.H: return h;
    
    stack=layout.stacks[i]
    all_stacks = True #True: all the bad located tops can be placed in stack
    
    for k in range(len(layout.stacks)):
        if k==i: continue
        if layout.is_sorted_stack(k): continue
            
        stack_k=layout.stacks[k]
        unsorted = len(stack_k)-layout.sorted_elements[k]
        prev = 1000;
        for j in range (1,unsorted+1):
            if stack_k[-j] <= prev and stack_k[-j] <=top:
                h += 1
                if h==layout.H: return h
                prev = stack_k[-j]
            else: 
                if j==1: all_stacks=False
                break
                
    if all_stacks: return layout.H
    else: return h

# Generador de layouts

In [78]:
import random 
def generate_random_layout(S,H,N):
    stacks = []
    for i in range(S):
        stacks.append([])
    
    for j in range(N):
        s=random.randint(0,S-1);
        while len(stacks[s])==H: s=s=random.randint(0,S-1);
        stacks[s].append(random.randint(1,N));
    

    return Layout(stacks,H)

# Greedy functions
![image](https://i.imgur.com/t4l142S.png)

In [79]:
def is_valid_BG_move(layout, s_o, s_d):
    if s_o != s_d  and len(layout.stacks[s_o]) > 0 \
    and  len(layout.stacks[s_d]) < layout.H \
    and layout.is_sorted_stack(s_o)==False \
    and layout.is_sorted_stack(s_d)==True \
    and layout.gvalue(s_o) <= layout.gvalue(s_d):
      return True

    else: return False

def select_bg_move(layout):
  bg_move = None
  S=len(layout.stacks)
  min_diff = 100
  for s_o in range(S):
     for s_d in range(S):
       if is_valid_BG_move(layout, s_o, s_d):
          diff = layout.gvalue(s_d)-layout.gvalue(s_o)
          if min_diff > diff:
            min_diff = diff
            bg_move = (s_o,s_d)
  return bg_move

# ...


Obtiene matriz a partir de Layout

In [80]:
import numpy as np

def get_ann_state(layout):
  S=len(layout.stacks)
  b = 20*np.ones([S,layout.H])
  for i,j in enumerate(layout.stacks):
     b[i][layout.H-len(j):] = j  
     #b[i][:len(j)] = j  
  b.shape=(S,layout.H)
  return b

layout = generate_random_layout(5,5,10)
print(layout.stacks)
get_ann_state(layout)


[[9, 5, 5], [8], [6], [10, 8, 7, 6], [2]]


array([[20., 20.,  9.,  5.,  5.],
       [20., 20., 20., 20.,  8.],
       [20., 20., 20., 20.,  6.],
       [20., 10.,  8.,  7.,  6.],
       [20., 20., 20., 20.,  2.]])

In [81]:
from copy import deepcopy
copy_lay = deepcopy(layout)
copy_lay.stacks

[[9, 5, 5], [8], [6], [10, 8, 7, 6], [2]]

El resto de las funciones se pueden adaptar de [aquí](https://github.com/rilianx/cpmp/blob/main/src/Greedy.cpp).

* `reduction_move`: es el movimiento de recucción. Hay que adaptarlo sin la parte de `smart_assignation`
* `get_destination_stack`: Es la función que en el paper se llama `select_destination`. Utiliza la función `ev_dest_stack` ar evaluar los stacks y retorna el mejor evaluado.
* `iter_greedy`: corresponde a una la iteración del algoritmo FRG


# Examples

In [82]:
layout = generate_random_layout(5,5,10)
print(layout.stacks)

[[1, 5], [6, 8], [7, 5], [], [2, 5, 1, 6]]


In [83]:
from copy import deepcopy
def greedy(layout):
  steps = 0
  while layout.unsorted_stacks>0:
    bg_move=select_bg_move(layout)
    if bg_move is not None:
      #print("bg_move:", bg_move)
      layout.move(bg_move)
    else:
      #print("no hay movimiento BG posibles")
      #print("elementos mal ubicados:", layout.total_elements - sum(layout.sorted_elements))
      return -1 # no lo resuelve
    steps +=1

  if layout.unsorted_stacks==0: 
    #print("resuelto!")
  
   return steps

layout = generate_random_layout(5,5,10)
copy_lay = deepcopy(layout)
copy_lay.stacks
val=greedy (layout)
listeichon = posibilidades(layout,copy_lay,val)
print(listeichon)

[-4, 1, 1, -1, 2, 1, 2, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1]


In [84]:
def posibilidades(layout,layoutt,num):
  lista=[]
  for i in range(5):
    for j in range(5):
      if(i!=j):
       layout.move((i,j))
       val=greedy(layout)
       if(val>-1):
         val=val+1-num
      
       lista.append(val)
       layout = deepcopy(layoutt)
       layout.stacks

  return lista
   

In [85]:
layout.stacks

[[1], [10, 8, 7, 5, 4], [5, 5], [2], [9]]

In [86]:
layout.move((1,0))

4

In [87]:
def generate_data(S=5, H=5, N=15, M=1, sample_size=1000, levels=1, x = [], y= []):
  lays = []
  for n in range(sample_size):
    layout = generate_random_layout(S,H,N)
    copy_lay = deepcopy(layout)
    copy_lay.stacks
    val=greedy (layout)
    listeichon = posibilidades(layout,copy_lay,val)
    x.append(get_ann_state(layout))
    y.append(listeichon)
    lays.append(layout)
  return x, y, lays

x,y,_= generate_data(sample_size=2, M=1, x = [], y= [])
x[0].shape=(5,5)
x[1].shape=(5,5)
print(x,y)

[array([[20., 20., 20.,  9.,  7.],
       [20., 20., 14.,  9.,  7.],
       [20., 12.,  4., 12., 11.],
       [20., 20., 20.,  6.,  2.],
       [20., 12., 10., 10., 10.]]), array([[20., 20., 20.,  7.,  4.],
       [20., 20., 20.,  7., 14.],
       [12., 10.,  9.,  9.,  8.],
       [20., 20., 20., 11., 15.],
       [20., 15., 15.,  7.,  3.]])] [[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

def generate_model(S=5, H=5, N=15):
  model = tf.keras.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(S*H,)))
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dense(20))
  return model

x_train, y_train, lays = generate_data(sample_size=10000, M=1, x=[], y=[])
unique, counts = np.unique(np.array(y_train), return_counts=True)

Fmodel=generate_model() # predice steps
Fmodel.compile(loss='mse', optimizer='adam', metrics=['mse'])
Fmodel.fit(np.array(x_train), np.array(y_train), epochs=5, verbose=True)

x_test, y_test, _ = generate_data(sample_size=10000, x=[], y=[])
test_loss, test_acc = Fmodel.evaluate(np.array(x_test), np.array(y_test))
print('Test accuracy:', test_acc)

In [ ]:
Fmodel.save_weights('model.h5')